# Data Load

In [1]:
import pandas as pd
import glob
import os

In [2]:
os.chdir(r"C:\Users\ManosIeronymakisProb\OneDrive - Probability\Bureaublad\thesis temporary\PRO\MI4\CMPNY_ESG_CORE\W365_UDAI")
# Load the slow data
slow_data_path = "S&P500_Slow_ESG_Scores_With_PermIDs.csv"
slow_df = pd.read_csv(slow_data_path)

# Define the folder path containing the text files
folder_path = r"C:\Users\ManosIeronymakisProb\OneDrive - Probability\Bureaublad\thesis temporary\PRO\MI4\CMPNY_ESG_CORE\W365_UDAI\*.txt"

# Initialize an empty list to store filtered data
filtered_data = []

# Iterate through each text file
for file_path in glob.glob(folder_path):
    # Read the data from the current file into a DataFrame
    fast_df = pd.read_csv(file_path, delimiter='\t')
    # Filter the fast data based on Company_Name matching with slow data
    filtered_df = fast_df[fast_df['assetCode'].isin(slow_df['PermID'])]
    filtered_data.append(filtered_df)

# Concatenate the filtered data into a single DataFrame
concatenated_df = pd.concat(filtered_data)

C:\Users\ManosIeronymakisProb\AppData\Local\Temp\ipykernel_19560\1741754162.py:15: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  fast_df = pd.read_csv(file_path, delimiter='\t')


# Creation of S&P500 companies ESG slow and fast anualized scores dataframe

In [3]:
# Extract year from 'windowTimestamp' column
concatenated_df['Year'] = concatenated_df['windowTimestamp'].str[:4]

# Convert 'Year' column to numeric
concatenated_df['Year'] = pd.to_numeric(concatenated_df['Year'])

# Calculate weighted mean ESG score for each year (Repeat this process for E,S,G,ESGCombined and ESGControversies)
weighted_mean_df = concatenated_df.groupby(['assetCode', 'Year']).apply(lambda x: (x['ESG'] * x['buzz']).sum() / x['buzz'].sum()).reset_index(name='Weighted_Mean_ESG')

# Pivot the slow data to have years as columns
slow_pivot = slow_df.melt(id_vars=['Index', 'Id', 'PermID'], var_name='Year', value_name='Slow_ESG')

# Pivot the weighted mean ESG scores to have years as columns
weighted_mean_pivot = weighted_mean_df.pivot(index='assetCode', columns='Year', values='Weighted_Mean_ESG')

# Convert index column to a normal column
weighted_mean_pivot.reset_index(inplace=True)

# Create a new list to store merged rows
merged_rows = []

# Create a new list to store exception rows
exception_rows = []

# Loop through Company_Name column of slow_df
for company_name in slow_df['PermID']:
    # Check if the company exists in weighted_mean_pivot
    if company_name in weighted_mean_pivot['assetCode'].values:
        # Get the index of the company in slow_df
        slow_index = slow_df.index[slow_df['PermID'] == company_name][0]
        # Get the index of the company in weighted_mean_pivot
        weighted_index = weighted_mean_pivot.index[weighted_mean_pivot['assetCode'] == company_name][0]
        # Merge the rows from slow_df and weighted_mean_pivot
        merged_row = pd.concat([slow_df.loc[slow_index], weighted_mean_pivot.loc[weighted_index]], ignore_index=True)
        # Rename the columns to mark the source
        merged_row.index = slow_df.columns.tolist() + ['{}F'.format(year) for year in merged_row.index[len(slow_df.columns):]]
        # Append the merged row to merged_rows
        merged_rows.append(merged_row)
    else:
        # Add the row from slow_df to exception_rows
        exception_rows.append(slow_df[slow_df['PermID'] == company_name])

# Convert merged_rows list to DataFrame
merged_df = pd.concat(merged_rows, axis=1).transpose()

# Set the column names after merging
merged_df.columns = list(slow_df.columns) + ['{}F'.format(year) for year in weighted_mean_pivot.columns]

# Concatenate exception_rows into a single dataframe
exceptions_df = pd.concat(exception_rows, ignore_index=True)

# Remove the 'Index' column from merged_df
merged_df.drop(columns=['Index'], inplace=True, errors='ignore')

# Remove the 'nameF' column from merged_df
merged_df.drop(columns=['nameF'], inplace=True, errors='ignore')

# Add 'S' to the end of each year in the slow columns
for column in merged_df.columns:
    column_str = str(column)
    if column_str.isdigit() and 'F' not in column_str:
        merged_df.rename(columns={column: '{}S'.format(column_str)}, inplace=True)


In [ ]:
merged_df.to_csv('merged_data_ESG_with_PermIDs.csv', index=False)

In [4]:
print(merged_df)

         PermID       Id                    Company_Name      2024S  \
0    4295903261    AFL.N                       Aflac Inc  50.066051   
1    4295903289    AES.N                        AES Corp  73.899905   
2    4295903265    ABT.N             Abbott Laboratories  83.319176   
3    4295905431  ADBE.OQ                       Adobe Inc  69.495684   
4    4295903297   AMD.OQ      Advanced Micro Devices Inc  67.830529   
..          ...      ...                             ...        ...   
489  5081500884   CEG.OQ       Constellation Energy Corp   54.09183   
490  5083504147  GEHC.OQ  GE Healthcare Technologies Inc  37.808184   
491  5083557475   KVUE.N                      Kenvue Inc        NaN   
492  5086711237   VLTO.N                    Veralto Corp        NaN   
493  5086720112    GEV.N                  GE Vernova Inc        NaN   

         2023S      2022S      2021S      2020S      2019S      2018S  ...  \
0    53.957505  53.152233  59.246564  54.841113  60.446906   51.54329